In [1]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 128  # the generated image resolution
    train_batch_size = 32
    eval_batch_size = 32  # how many images to sample during evaluation
    num_epochs = 500
    patience = 20  # early stopping patience
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "ddpm-butterflies-128"  # the model name locally and on the HF Hub



config = TrainingConfig()

In [2]:
import torch
from torch.utils.data import Dataset, ConcatDataset
import numpy as np

class GaussianDataset(Dataset):
    def __init__(self, num_samples, input_dim, cov, samples_per_obs):
        self.num_samples = num_samples
        self.input_dim = input_dim
        self.cov = cov

        self.o = []
        self.a = []

        # Generate input data (o)
        for _ in range(num_samples):
            obs = np.random.normal(size=(input_dim))

            for _ in range(samples_per_obs):

                # Generate output data (a) using the Gaussian distribution
                act = np.random.multivariate_normal(mean=obs, cov=self.cov)

                self.o.append(obs)
                self.a.append(act)

        print(f"Generated {len(self.o)} samples")


    def __len__(self):
        return len(self.o)
    
    def get_observations(self):
        return self.o

    def __getitem__(self, idx):
        o = torch.tensor(self.o[idx], dtype=torch.float32)
        a = torch.tensor(self.a[idx], dtype=torch.float32)
        return o, a

In [ ]:
# Set parameters
real_samples = 100
sim_saples = 10000
action_dim = 2
cov = np.array([[1.0, 0.5], [0.5, 1.0]])
sim_cov = cov + np.random.normal(0, 0.1, cov.shape)
samples_per_obs = 10

# Create dataset
real_dataset = GaussianDataset(real_samples, action_dim, cov, samples_per_obs)

sim_dataset = GaussianDataset(sim_saples, action_dim, sim_cov, samples_per_obs)

# Concatenate datasets
dataset = ConcatDataset([real_dataset, sim_dataset])

# Create data loader
batch_size = 32
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for step, batch in enumerate(data_loader):
    o, a = batch
    print(o.shape, a.shape)
    break

# Construct validation dataset


Generated 100 samples
Generated 10000 samples
torch.Size([32, 2]) torch.Size([32, 2])


In [4]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, action_dim, hidden_dim=64):
        super(MLP, self).__init__()
        input_dim = 2 * action_dim + 1

        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, action_dim)

    def forward(self, o, a, timestep):
        # Add a new dimension to timestep to match the batch dimension of o and a
        timestep = timestep.unsqueeze(-1)  # (32) -> (32, 1)
        
        # Concatenate o, a, and timestep along the last dimension
        x = torch.cat([o, a, timestep], dim=-1)  # (32, 2), (32, 2), (32, 1) -> (32, 5)
        
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the MLP with the desired action dimension
model = MLP(action_dim)

# Print the MLP architecture
print(model)

MLP(
  (fc1): Linear(in_features=5, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)


In [8]:
o, a = dataset[0]
sample_inp = torch.cat([o, a, torch.tensor([0])])
print("Input shape:", sample_inp.shape)

# Forward pass
#output = model(o, a, torch.tensor([0]))
#print("Output shape:", output.shape)


Input shape: torch.Size([5])


In [10]:
from diffusers import DDPMScheduler
import torch.nn.functional as F

noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
noise = torch.randn(a.shape)
timesteps = torch.LongTensor([50])
noisy_image = noise_scheduler.add_noise(a, noise, timesteps)

observation = torch.tensor([0.0, 0.0])

print("Noisy image shape:", noisy_image.shape)
print("Noisy image:", noisy_image)


#noise_pred = model(noisy_image, observation, timesteps)
#loss = F.mse_loss(noise_pred, noise)

Noisy image shape: torch.Size([2])
Noisy image: tensor([-0.6609,  1.9509])


In [11]:
from diffusers.optimization import get_cosine_schedule_with_warmup

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(data_loader) * config.num_epochs),
)

In [6]:
from accelerate import Accelerator
from huggingface_hub import create_repo, upload_folder
from tqdm.auto import tqdm
from pathlib import Path
import os
from diffusers import DDPMPipeline
from diffusers.utils import make_image_grid
import os
import wandb


wandb.init(project="diffusion-toy", entity="rohanb27-csail")

def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize accelerator and tensorboard logging
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="wandb",
        project_dir=os.path.join(config.output_dir, "logs"),
    )
    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)

    # Prepare everything
    # There is no specific order to remember, you just need to unpack the
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    epochs_since_improvement = 0

    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            # Unpack the batch
            obs, act = batch
            # Concatenate observation and action
            inputs = torch.cat((obs, act), dim=1)
            bs = inputs.shape[0]

            # Sample noise to add to the actions
            noise = torch.randn(act.shape, device=act.device)

            # Sample a random timestep for each action
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=act.device,
                dtype=torch.int64
            )

            # Add noise to the clean actions according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_actions = noise_scheduler.add_noise(act, noise, timesteps)

            
            with accelerator.accumulate(model):
                # Predict the noise residual
                noise_pred = model(obs, noisy_actions, timesteps)[0]
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

                # After each epoch you optionally sample some demo images with evaluate() and save the model
        if accelerator.is_main_process:
            pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                pipeline.save_pretrained(config.output_dir)

/home/rohanb/DiffusionToy/diffusion_toy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Currently logged in as: rohanb27 (rohanb27-csail). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [12]:
from accelerate import notebook_launcher
args = (config, model, noise_scheduler, optimizer, data_loader, lr_scheduler)
notebook_launcher(train_loop, args, num_processes=1)

Launching training on CPU.


Epoch 0:   0%|          | 0/316 [00:00<?, ?it/s]/tmp/ipykernel_47048/2673571491.py:64: UserWarning: Using a target size (torch.Size([32, 2])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(noise_pred, noise)
Epoch 0:   1%|          | 2/316 [00:00<00:02, 115.89it/s, loss=241, lr=4e-7, step=1]

Epoch 0: 100%|█████████▉| 315/316 [00:01<00:00, 274.52it/s, loss=1.24, lr=6.3e-5, step=314]  /tmp/ipykernel_47048/2673571491.py:64: UserWarning: Using a target size (torch.Size([20, 2])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(noise_pred, noise)
Epoch 28: 100%|██████████| 316/316 [00:01<00:00, 255.46it/s, loss=0.74, lr=3.97e-5, step=9163] 
self.unet=MLP(
  (fc1): Linear(in_features=5, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
) of type <class '__main__.MLP'> cannot be saved.
Epoch 48: 100%|██████████| 316/316 [00:01<00:00, 266.36it/s, loss=0.784, lr=1.05e-7, step=15483]
self.unet=MLP(
  (fc1): Linear(in_features=5, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2